In [1]:
require 'nn'
require 'cudnn'
require 'cunn'
require 'cutorch'
DataLoader = require 'dataloader2'
require 'CicularShift'
require 'optim'

In [2]:
nInput = 49284
nFeature = 120
batchSize = 1024
nSamples = 1024
nTrial = 1
offsetRatio = 0.3
trainLoader = DataLoader(
    '/home/chenxi/dataset/dssm/query.test.tsv', 
    '/home/chenxi/dataset/dssm/doc.test.tsv', 
    batchSize, 
    nSamples, 
    nInput)

In [3]:
model = nn.Sequential()
model:add(nn.Linear(nInput, 400))
model:add(cudnn.ReLU())


model:add(nn.Linear(400, nFeature))
model:add(nn.View(-1, batchSize, nFeature))
model:add(nn.SplitTable(1))

par = nn.ParallelTable()

concat1 = nn.Concat(1)
for i=1, nTrial+1 do
    concat1:add(nn.Identity())
end
concat2 = nn.Concat(1)
concat2:add(nn.Identity())

for i=1, nTrial do
    base = math.floor(offsetRatio * batchSize)
    offset = base + torch.random(batchSize - 1 - base)
    concat2:add(nn.CicularShift(1, offset, batchSize))
end
par:add(concat1)
par:add(concat2)

model:add(par)
model:add(nn.CosineDistance())

model:add(nn.View(-1, batchSize))
model:add(nn.Transpose({1, 2}))
-- model:add(nn.MulConstant(20))

-- weights initialization
local function LinearInit(name)
    for k, v in pairs(model:findModules(name)) do
        local n = v.weight:size(1) + v.weight:size(2)
        v.weight:normal(0, math.sqrt(4 / n))
        v.bias:fill(0)
    end
end
LinearInit('nn.Linear')
LinearInit('nn.SparseLinear')

model:cuda()
criterion = nn.CrossEntropyCriterion():cuda()
-- criterion = nn.CosineEmbeddingCriterion():cuda()
params, gradParams = model:getParameters()

In [ ]:
counter = 0
function feval(x_new) 
    return criterion.output, gradParams
end

sgd_config = {
    learningRate = 0.001
}

for i=1,7 do
    for sample in trainLoader:nextBatch() do
        input = torch.CudaTensor()
        target = torch.CudaTensor()
        input:resize(sample.input:size()):copy(sample.input)
        target:resize(sample.target:size()):copy(sample.target)    

        output = model:forward(input)

        loss = criterion:forward(output, target)
        print(loss)

        model:zeroGradParameters()
        model:backward(input, criterion:backward(output, target))
        optim.sgd(feval, params, sgd_config)
        
        w = model:get(1).weight
        grad = model:get(1).gradWeight * 0.0001
        w_scale = torch.norm(w)
        grad_scale = torch.norm(grad)
        print('update', grad_scale / w_scale)
        counter = counter + 1
    end
end

0.60440319776535	


update	0.046334960028864	


0.69313228130341	


update	6.9981631392934e-06	


0.69313228130341	


update	6.9982106247983e-06	


0.69313222169876	


update	6.9980542879052e-06	
